In [1]:
import scipy.io as sio
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.integrate import simps
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import FastICA
from sklearn import preprocessing
from scipy import signal

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset as torchDataset



In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
data_dir = '/home/ghn8/courses/neuroengineering/project/data'



In [3]:
def get_patient_training_data(pid):
    ICTAL = 1
    NONICTAL = 0
    
    ictal_data_dir = '/home/ghn8/courses/neuroengineering/project/data/patient_%d/ictal train/' % pid
    nonictal_data_dir = '/home/ghn8/courses/neuroengineering/project/data/patient_%d/non-ictal train/' % pid
    
    train_inputs = []
    train_labels = []
    
    for i in range(1, len(os.listdir(ictal_data_dir))+1):
        file_path = os.path.join(ictal_data_dir, 'patient_%d_%d.mat' % (pid, i))
        data_mat = sio.loadmat(file_path)
        data = data_mat['data'].astype(np.float32)
        if(np.isnan(data).any()):
            raise(ValueError("NaN in ictal data"))
        train_inputs.append(data.T) # transpose data to get num_channels x timepoints
        train_labels.append(ICTAL)
    
    for i in range(1, len(os.listdir(nonictal_data_dir))+1):
        file_path = os.path.join(nonictal_data_dir, 'patient_%d_%d.mat' % (pid, i))
        data_mat = sio.loadmat(file_path)
        data = data_mat['data'].astype(np.float64)
        if(np.isnan(data).any()):
            raise(ValueError("NaN in non-ictal data"))
        train_inputs.append(data.T) # transpose data to get num_channels x timepoints
        train_labels.append(NONICTAL)
        
    train_inputs = np.asarray(train_inputs)
    train_labels = np.asarray(train_labels)
    
    return train_inputs, train_labels

In [4]:
def get_patient_test_data(pid):
    test_dir = '/home/ghn8/courses/neuroengineering/project/data/patient_%d/test/' % pid
    
    test_inputs = []
    
    for i in range(1, len(os.listdir(test_dir))+1):
        file_path = os.path.join(test_dir, 'patient_%d_test_%d.mat' % (pid, i))
        if os.path.exists(file_path):
            data_mat = sio.loadmat(file_path)
            data = data_mat['data'].astype(np.float32)
            data = np.nan_to_num(data)
            if(np.isnan(data).any()):
                raise(ValueError("NaN in test data"))
            test_inputs.append(data.T) # transpose data to get num_channels x timepoints
        
    test_inputs = np.asarray(test_inputs)
    
    return test_inputs

In [5]:
NUM_PATIENTS = 7
all_train_data = {}

train_data_dump_file = os.path.join('/home/ghn8/courses/neuroengineering/project/preprocessed/all_data', 'all_train_data.pkl')

if not os.path.exists(train_data_dump_file):
    for pid in range(1, NUM_PATIENTS+1):
        train_inputs, train_labels = get_patient_training_data(pid)
        all_train_data['patient_%d' % pid] = { 'inputs': train_inputs, 'labels': train_labels}
        print('Finished reading data for patient %d' % pid)

    pickle.dump(all_train_data, open(train_data_dump_file, 'wb'))
else:
    all_train_data = pickle.load(open(train_data_dump_file, 'rb'))


In [6]:
all_test_data = {}

test_data_dump_file = os.path.join('/home/ghn8/courses/neuroengineering/project/preprocessed/all_data', 'all_test_data.pkl')

if not os.path.exists(test_data_dump_file):
    for pid in range(1, NUM_PATIENTS+1):
        test_inputs = get_patient_test_data(pid)
        all_test_data['patient_%d' % pid] = test_inputs
        print('Finished reading test data for patient %d' % pid)

    pickle.dump(all_test_data, open(test_data_dump_file, 'wb'))
else:
    all_test_data = pickle.load(open(test_data_dump_file, 'rb'))


In [7]:
def fft(timeseries):
    return np.log10(np.absolute(np.fft.rfft(timeseries)))

def get_spectral_features(timeseries):
    raw_fft = fft(timeseries)
    mask = np.logical_or(np.isneginf(raw_fft), np.isinf(raw_fft))
    raw_fft[mask] = 0.0;

    return raw_fft
    
    

In [8]:
def shuffle_data(features, labels):
    assert features.shape[0] == labels.shape[0], "Mismatched number of samples"
    shuffled_indices = np.random.permutation(features.shape[0])
    
    shuffled_features = features[shuffled_indices, :]
    shuffled_labels = labels[shuffled_indices]
    return shuffled_features, shuffled_labels


In [9]:
def transform_inputs_to_spectral_features(inputs):
    num_samples = inputs.shape[0]
    features = []

    for i in range(num_samples):
        sample = inputs[i, :, :].squeeze()

        spectral_features = get_spectral_features(sample)
        features.append(spectral_features)

    features = np.asarray(features)
    return features


In [10]:
def ica_clean(data, n_components):
    ica = FastICA(n_components=n_components)
    return ica.fit_transform(data)

In [11]:
def get_temporal_features(inputs, num_eigenvalues=-1, num_icas=0):
    num_samples = inputs.shape[0]
    features = []

    for i in range(num_samples):
        sample = inputs[i, :, :].squeeze()
        
        if num_icas > 0:
            sample = ica_clean(sample, num_icas)
        
        norm_sample = preprocessing.scale(sample)
        corr = np.corrcoef(norm_sample)
        
        uppper_triag_indices = np.triu_indices(corr.shape[0], 1)
        upper_triag_vals = corr[uppper_triag_indices].ravel()
        
        eigenvalues, _ = np.linalg.eig(corr)
        eigenvalues = np.sort(eigenvalues)
        
        if num_eigenvalues > 0:
            eigenvalues = eigenvalues[:num_eigenvalues]

        features.append(np.concatenate((upper_triag_vals, eigenvalues)))
    features = np.asarray(features)
    return features


In [12]:
def transform_inputs_to_features(inputs):
    spectral_features = transform_inputs_to_spectral_features(inputs, lpf_freq)
        
    temporal_features = get_temporal_features(inputs)
    
    features = np.concatenate((spectral_features, spectral_features), axis=1)
    
    return features


In [13]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Convolutional neural network
class ConvNet_Spectral20k(nn.Module):
    def __init__(self, num_in_channels):
        super(ConvNet_Spectral20k, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(num_in_channels, 32, kernel_size=5),
            # nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5))
        self.layer2 = nn.Sequential(
            nn.Conv1d(32, 16, kernel_size=5),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5))
        self.fc = nn.Linear(99*16, 1)
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # print(self.num_flat_features(out))
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
class ConvNet_Spectral2k(nn.Module):
    def __init__(self, num_in_channels):
        super(ConvNet_Spectral2k, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(num_in_channels, 32, kernel_size=5),
            # nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5))
        self.layer2 = nn.Sequential(
            nn.Conv1d(32, 16, kernel_size=5),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5))
        self.fc = nn.Linear(9*16, 1)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # print(self.num_flat_features(out))
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
class EEGDataset(torchDataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        features = self.features[index, :,  :].astype('float')
        label = np.expand_dims(self.labels[index], axis=1)

        return features, label

    def __len__(self):
        return self.features.shape[0]

def train(train_loader, num_epochs, num_features, num_in_channels, learning_rate):
    if num_features > 300:
        model = ConvNet_Spectral20k(num_in_channels)
        thresh = 5e-3
    else:
        model = ConvNet_Spectral2k(num_in_channels)
        thresh = 1e-3
    model = model.to(device)

    # Loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # Training
    total_step = len(train_loader)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        epoch_loss = epoch_loss / len(train_loader)
        if ((epoch+1) % 10 == 0):
            print("\tEpoch [%d/%d], Loss: %.4f" % (epoch+1, num_epochs, epoch_loss))
        if epoch_loss <= thresh:
            break

    return model

def predict(model, test_loader):
    # Test
    model.eval()
    predicted_probs = np.empty((0, 1))
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device, dtype=torch.float)

            probs = torch.sigmoid(model(inputs)).data.cpu().numpy()
            predicted_probs = np.concatenate((predicted_probs, probs))
    
    return predicted_probs

def eval(model, test_loader):
    # Test
    model.eval()
    
    all_predicted_labels = np.empty((0, 1))
    all_probs = np.empty((0, 1))

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            probs = torch.sigmoid(model(inputs)).data.cpu().numpy()
            predicted_labels = np.where(probs > 0.5, 1, 0)

            all_predicted_labels = np.concatenate((all_predicted_labels, predicted_labels))
            all_probs = np.concatenate((all_probs, probs))
    
    return all_predicted_labels.squeeze(), all_probs.squeeze()


In [ ]:
output_file = os.path.join('/home/ghn8/courses/neuroengineering/project/gia/outputs', 'cnn_spectral_2layer32-16_decay1e-4_epoch5k.csv')
is_testing = True

for pid in range(1, NUM_PATIENTS+1):
    train_inputs = all_train_data['patient_%d' % pid]['inputs']
    train_labels = all_train_data['patient_%d' % pid]['labels']
    test_inputs = all_test_data['patient_%d' % pid]

    train_features = transform_inputs_to_spectral_features(train_inputs)
    test_features = transform_inputs_to_spectral_features(test_inputs)
    
    # Hyperparameters
    num_epochs = 5000
    batch_size = 200
    learning_rate = 0.001
    num_samples, num_channels, num_features = train_features.shape

    
    if is_testing:
        print("Predicting patient %d" % pid)
        
        if pid == 1:
            fh = open(output_file, 'w')
            fh.write("id,prediction\n")
        else:
            fh = open(output_file, 'a')
            
        train_dataset = EEGDataset(features=train_features,
                                  labels=train_labels)
        test_dataset = EEGDataset(features=test_features,
                                  labels=np.zeros((test_features.shape[0], 1)))

        # Data loader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                  batch_size=batch_size,
                                                  shuffle=False)
        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=False)


        model = train(train_loader, num_epochs, num_features, num_channels, learning_rate)
        predicted_probs = predict(model, test_loader)

        for i in range(len(predicted_probs)):
            fh.write("patient_%d_%d,%f\n" % (pid, i+1, predicted_probs[i]))
        fh.close()
    else:
        print("Evaluating patient %d" % pid)
        val_fraction = 0.2

        # creat training and validation data
        [shuffled_features, shuffled_labels] = shuffle_data(train_features, train_labels)

        num_val_samples = int(num_samples * val_fraction)

        val_features = shuffled_features[:num_val_samples, :]
        val_labels = shuffled_labels[:num_val_samples]

        bootstraped_train_features = shuffled_features[num_val_samples:, :]
        bootstraped_train_labels = shuffled_labels[num_val_samples:]
        
        train_dataset = EEGDataset(features=bootstraped_train_features,
                                  labels=bootstraped_train_labels)
        test_dataset = EEGDataset(features=val_features,
                                  labels=val_labels)

    
        # Data loader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                  batch_size=batch_size,
                                                  shuffle=False)
        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=False)


        model = train(train_loader, num_epochs, num_features, num_channels, learning_rate)
        all_predicted_labels, all_probs = eval(model, test_loader)
        accuracy = (all_predicted_labels == val_labels).sum() * 100.0 / val_labels.shape[0]
        
        print("Accuracy: %.2f" % accuracy + "%")
        print("AUC :", roc_auc_score(val_labels, all_probs))

